##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hand Landmarks Detection with MediaPipe Tasks

This notebook shows you how to use MediaPipe Tasks Python API to detect hand landmarks from images.

## Preparation

Let's start with installing MediaPipe.

In [1]:
!pip install -q mediapipe

# Test 1 - 2D pred

In [2]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

In [4]:
import mediapipe as mp
import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

# Use OpenCV’s VideoCapture to load the input video:
vid_path = './Camera_LF.avi'
video_capture = cv2.VideoCapture(vid_path)

# Load the frame rate of the video using OpenCV’s CV_CAP_PROP_FPS:
frame_rate = video_capture.get(cv2.CAP_PROP_FPS)

# calculate the timestamp for each frame in ms:
timestamp = 1 / frame_rate * 1000

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

model_path = '/Users/alighavampour/Desktop/Projects/Tracker/hand_landmarker.task'

annotated_results = []
# Create a hand landmarker instance with the video mode:
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO)
with HandLandmarker.create_from_options(options) as landmarker:
    frame_timestamp = 0
    # Loop through each frame in the video using VideoCapture.read():
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break
        
        # Convert the frame received from OpenCV to a MediaPipe’s Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

        # calculate the frame timestamp in ms:
        frame_timestamp += timestamp

        # Perform hand landmarks detection on the provided single image:
        hand_landmarker_result = landmarker.detect_for_video(mp_image, int(frame_timestamp))

        annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), hand_landmarker_result)
        # cv2.imshow('Frame',annotated_image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows() # It destroys the image showing the window.

        # append the annotated image to the list:
        annotated_results.append(annotated_image)

# Release the VideoCapture object and close the OpenCV window:
video_capture.release()

# Save the annotated results as a video:
output_path = './output.avi'
height, width, _ = annotated_results[0].shape
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, frame_rate, (width, height))
for frame in annotated_results:
    out.write(frame)

out.release()

        

I0000 00:00:1715120637.067141 4269195 gl_context.cc:357] GL version: 2.1 (2.1 ATI-4.8.101), renderer: AMD Radeon R9 M395 OpenGL Engine
